In [1]:
import pymongo
import psycopg2
import keys

""" 
This notebook was written to migrate data from MongoDB to Postgres.
It assumes that the target server is already initialized.
It also assumes you have appropriately configured MongoClient in your environment to connect to the MongoDB deployment
"""

mongo = pymongo.MongoClient(keys.mongodb_cs)
print('Mongo connection')
pgconn = psycopg2.connect(
    host="localhost",
    dbname='caroldb',
    user='postgres',
    password=keys.pg_pwd
)
print('PG connection')

with pgconn.cursor() as pgcur:
    
    insert_query = """
    INSERT INTO linktable (filingname, filingdate, aenumber, attachment, hyperlink)
    VALUES (%s, %s, %s, %s, %s)
    """
    batch_size = 1000
    batch_data = []

    # Loop through linktable in Mongo, inserting batch to PG
    for document in mongo['Transcripts']['linktable'].find():
        filing_name = document.get("FilingName", None)
        filing_date = document.get("FilingDate", None)
        ae_number = document.get("AENumber", None)
        attachment = True if document.get("Attachment") == "Attachment" else False
        hyperlink = document.get("Hyperlink", None)

        batch_data.append((filing_name, filing_date, ae_number, attachment, hyperlink))

        # at batch_size, insert to PG and reset batch
        if len(batch_data) == batch_size:
            pgcur.executemany(insert_query, batch_data)
            pgconn.commit()
            batch_data = []

    # Insert any remaining linktable data that didn't fill final batch
    if batch_data:
        pgcur.executemany(insert_query, batch_data)
        pgconn.commit()

    # prepare insert command for pages
    insert_query = """
    INSERT INTO pages (embedding, page_number, parent_doc, text)
    VALUES (%s, %s, %s, %s)
    """

    # pages is too large to create a mongo cursor covering the whole table
    # instead, we iterate through pages batchwise using the hyperlink/parent_doc as a batch criterion
    for row in mongo['Transcripts']['transcripts'].find(projection={"Hyperlink":1, "_id":0}):
        hyper = row.get('Hyperlink')
        document_name = hyper[hyper.find('KSM2/')+5:]
        print(f'Preparing {document_name}')
        batch_data = []

        for document in mongo['Transcripts']['pages'].find(filter={"parent_doc":document_name}):
            embedding = document.get("embeddings", None)
            page_number = document.get("page_number", None)
            parent_doc = document.get("parent_doc", None)
            text = document.get("text", None)

            batch_data.append((embedding, page_number, parent_doc, text))

        pgcur.executemany(insert_query, batch_data)
        pgconn.commit()

    # prepare insert command for transcripts table
    insert_query = """
    INSERT INTO transcripts (filingname, filingdate, aenumber, attachment, hyperlink, summary)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    batch_data = []

    for document in mongo['Transcripts']['transcripts'].find():
        filing_name = document.get("FilingName", None)
        filing_date = document.get("FilingDate", None)
        ae_number = document.get("AENumber", None)
        attachment = True if document.get("Attachment") == "Attachment" else False
        hyperlink = document.get("Hyperlink", None)
        summary = document.get("Summaries", None)

        batch_data.append((filing_name, filing_date, ae_number, attachment, hyperlink, summary))

    pgcur.executemany(insert_query, batch_data)
    pgconn.commit()

pgconn.close()

Mongo connection
PG connection
Preparing KSM II (Trans17Dec2013-PM1).pdf
Preparing KSM II (KSM 806 Transcript 25 Feb 2016)_Part1.pdf
Preparing KSM II (Trans17Jun2019-MERGED).pdf
Preparing KSM II (TRANS11Dec2015-PM1).pdf
Preparing KSM II (TRANS19August2013-PM2).pdf
Preparing KSM II (TRANS12February2013-AM1).pdf
Preparing KSM II (TRANS21August2013-AM2).pdf
Preparing KSM II (TRANS10Sept2018-MERGED2).pdf
Preparing KSM II (KSM 806 Transcript 25 Feb 2016)_Part2.pdf
Preparing KSM II (TRANS26July2016-AM1).pdf
Preparing KSM II (TRANS19Oct2017-PM1).pdf
Preparing KSM II (TRANS30Oct2019-PM-MERGED).pdf
Preparing KSM II (Trans20Jun2019-MERGED).pdf
Preparing KSM II (TRANS21Mar2017-AM1).pdf
Preparing KSM II (TRANS23Jan2020-MERGED).pdf
Preparing KSM II (806 Transcript 23 MAR 2017).pdf
Preparing KSM II (TRANS22Feb2016-AM2).pdf
Preparing KSM II (TRANS11Feb2015)-AM1.pdf
Preparing 58416-KSM II (803TRANS6Oct2023).pdf
Preparing 58248-KSM II (TRANS22Sep2023-MERGED).pdf
Preparing KSM II (TRANS30May2016-AM1).pd